<a href="https://colab.research.google.com/github/danie1goodwin/uvadlc_practicals_2023/blob/main/Copy_of_DL1_Assignment2_Q1_1_draft_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DL1 Assignment2 - Q1.1 draft code

This is a small help from us to save you some coding. This notebook is **not** graded, you are free to edit it.

Further advise:
1. Start with File/Save a copy in Drive
2. Set GPU usage under Runtime/Change runtime type/Hardware accelerator.

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.8 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn
import timm
import numpy as np
from torchvision import models
from matplotlib import pyplot as plt
from typing import Callable


%matplotlib inline

In [ ]:
def vit_s_8():
    """ViT-S/8 is not a default torchvision model, so we provide it by timm"""
    # Accuracy approximation comes from
    # https://openreview.net/pdf?id=LtKcMgGOeLt
    # and DINO
    # https://arxiv.org/abs/2104.14294
    return timm.create_model('vit_small_patch8_224')

# Model definitions
# Optional Q: These are uncalled functions. What do you think would happen
# if we called all of them once? Why didn't we do that?
model_defs = [
    vit_s_8,
    models.vit_b_32,
    models.vgg11,
    models.vgg11_bn,
    models.resnet18,
    models.densenet121,
    models.mobilenet_v3_small,
]

# Accuracies per model
model_accs = {
    'vit_s_8': 80., # Approximated
    'vit_b_32' : 75.912,
    'vgg11' : 69.02,
    'vgg11_bn' : 70.37,
    'resnet18' : 69.758,
    'densenet121' : 74.434,
    'mobilenet_v3_small' : 67.668,
}


def measure_runtime_per_forward(model:nn.Module, no_grad:bool, batch_size:int=8):
    """Measures the time for a single pass in milliseconds"""

    # Generate fake RGB input (224x224)
    #######################
    # PUT YOUR CODE HERE  #
    #######################
    inp = torch.rand((batch_size,3,224,224), device = 'cuda')
    #######################
    # END OF YOUR CODE    #
    #######################

    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()

    # Run the model
    #######################
    # PUT YOUR CODE HERE  #
    #######################
    model(inp)
    #######################
    # END OF YOUR CODE    #
    #######################

    end.record()
    torch.cuda.synchronize()
    return start.elapsed_time(end)


def evaluate_model(model_def:Callable, no_grad:bool, batch_size:int=8):

    # Retreive initial memory allocation
    initial_vram = torch.cuda.memory_allocated()
    print(initial_vram)

    # Define model
    model = model_def().cuda().eval()
    # Access name as: model.__name__

    # Parameters that need to be filled
    n_params = None
    times, vrams = [], []
    mean_time = None
    mean_vram = None

    #######################
    # PUT YOUR CODE HERE  #
    #######################
    # Step 1: Calculate the number of **trainable** parameters
    n_params = 0
    for param in model.parameters():
      if param.requires_grad:
        n_params += param.numel()

    # Step 2: Warm up with a few passes
    for _ in range(5):
      input = torch.rand((batch_size,3,224,224), device = 'cuda')
      model(input)

    # Step 3: Run N forward passes and save the runtime +
    #         the vram allocated by the model
    for _ in range(15):
      torch.cuda.reset_peak_memory_stats()
      times.append(measure_runtime_per_forward(model, no_grad, batch_size))
      vrams.append(torch.cuda.memory_allocated(device = 'cuda')) # this is in B

    # Step 4: Take the mean, preferably with dropping possible outliers
    def exclude_outliers(lst):
      mean = np.mean(lst)
      std = np.std(lst)
      threshold = 2*std
      return [x for x in lst if np.abs(x-mean)<threshold]

    mean_time = np.mean(exclude_outliers(times))
    mean_vram = np.mean(exclude_outliers(vrams))

    #######################
    # END OF YOUR CODE    #
    #######################

    # Clean up space for the model
    del model
    torch.cuda.empty_cache()

    return mean_time, mean_vram, n_params


In [ ]:
#######################
# PUT YOUR CODE HERE  #
#######################
import matplotlib.pyplot as plt

# Make your plots here with matplotlib

# Example usage of the above functions:
for model_def in model_defs:
    name = model_def.__name__
    time, vram, n_params = evaluate_model(model_def, True, batch_size = 8)
    print(f'{name}  {time}ms  {vram} MB  number-parameters={n_params}')

#######################
# END OF YOUR CODE    #
#######################

14614665216


OutOfMemoryError: ignored